# Gaussian Naive Bayes
A reimplementation of the `sklearn` Gaussian Naive Bayes classifier. 

Provide a python class MyGaussianNB that implements Gaussian Naive Bayes. 
The API specification for sklearn classifiers is here: https://scikit-learn.org/stable/developers/develop.html 
Implement the ‘fit’ and ‘predict’ methods. 


In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.base import BaseEstimator, ClassifierMixin
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import GridSearchCV
from sklearn.impute import KNNImputer

## My GaussianNB
Reimplementation of a Gaussian Naive Bayes.

In [2]:
class MyGaussianNB(BaseEstimator, ClassifierMixin):          
    def fit(self, Xt, yt):
        self.var_smoothing = 1e-9   # zero variance will cause division by zero errors.
        self.Xt = Xt
        self.yt = yt
        self.n_feat = Xt.shape[1]
        self.mus = {}
        self.sig_sqs = {}
        self.priors = {}
        
        c_dict = Counter(self.yt)
        
        for c in c_dict.keys():
            self.mus[c] = np.zeros(self.n_feat) # where the means will be stored
            self.sig_sqs[c] = np.zeros(self.n_feat) # where the variances will be stored
            self.priors[c] = c_dict[c]/Xt.shape[0]
            
            mask = self.yt == c
            X_tr_c = self.Xt[mask, :] # the rows for this class label
            
            for f in range(self.n_feat):
                self.mus[c][f] = np.nanmean(X_tr_c[:,f]) #nanmean
                self.sig_sqs[c][f] = np.nanvar(X_tr_c[:,f] + self.var_smoothing)  #nanvar              
        #print(self.mus)
        #print(self.sig_sqs)
        
        return self
    
    # The predictions are the most common class in the training set.
    def predict(self, Xtes):
        self.Xtes = Xtes 
        res_list = []
        for sample in Xtes:
            res_list.append(self.predict_single(sample))
            
        return np.array(res_list)
    
    def predict_single(self, x_single):
        probs = {}
        for c in self.priors.keys():   # for each of the class labels
            probs[c] = self.priors[c]   # prior probability
            
            for i, f in enumerate(x_single): #Skip missing values
                if not np.isnan(f):
                    t1 = 1/math.sqrt(2*math.pi*self.sig_sqs[c][i])
                    num = (f - self.mus[c][i])**2
                    den = 2*self.sig_sqs[c][i]
                    pxi_y = t1 * math.exp(-num/den)   #calculate P(x/y)
                    probs[c] = probs[c] * pxi_y   #posterior = prior * likelihood
                    #print(t1, num, den, pxi_y)
                    #print(probs)
                #print(c, self.priors[c])
        return max(probs, key=probs.get) # Return the key with the largest value
        

## Testing
Test the performance of your implementation against the `GaussianNB` implementation in `scikit-learn`. Use a range of datasets for this testing.   
Four datasets are used for testing; testing on a hold out set:
 - **penguins**: check that mean and variance estimates are the same, check that predictions are the same. 
 - **diabetes**: check that predictions are the same.
 - **glassV2**: test that predictions are the same. 
 - **bike_sharing**: test that predictions are the same. 

The main component in the testing is to check the `fidelity` of the testing against the Gaussian Naive Bayes implementation in `scikit-learn`.    
The `fidelity_tests` function compares predictions across multiple runs (different holdout tests). It uses `accuracy_score` to do the comparison.

In [3]:
def fidelity_tests (X,y, nreps = 10):
    for rs in range(1, nreps + 1):
        X_tr_raw, X_ts_raw, y_train, y_test = train_test_split(X_raw, y, 
                                                               random_state=rs, 
                                                               test_size=1/2)
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_tr_raw)
        X_test = scaler.transform(X_ts_raw)
        gnb = GaussianNB()
        mgnb = MyGaussianNB()
        mgnb.fit(X_train,y_train)
        gnb.fit(X_train,y_train)
        ascore = accuracy_score(gnb.predict(X_test),mgnb.predict(X_test)) 
        gacc = accuracy_score(gnb.predict(X_test),y_test)
        macc = accuracy_score(mgnb.predict(X_test),y_test)
        print ("Run: %d Score: %.2f SK acc: %.2f My acc: %.2f" % (rs, ascore, gacc, macc))

### Penguins

In [4]:
penguins = pd.read_csv('penguins.csv', index_col = 0)
print(penguins.shape)
penguins.head()

f_names = ['bill_length_mm', 'bill_depth_mm','flipper_length_mm', 'body_mass_g']
penguins = penguins[f_names + ['species']]

(333, 8)


In [5]:
y = penguins.pop('species').values
X_raw = penguins.values
X_tr_raw, X_ts_raw, y_train, y_test = train_test_split(X_raw, y, random_state=2, test_size=1/2)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_tr_raw)
X_test = scaler.transform(X_ts_raw)
max_k = X_train.shape[1]
X_train.shape, X_test.shape

((166, 4), (167, 4))

**Model Parameters**  
The means are the same:

In [6]:
gnb = GaussianNB()
gnb.fit(X_train,y_train)
mgnb = MyGaussianNB()
mgnb.fit(X_train,y_train)

MyGaussianNB()

In [7]:
mgnb.sig_sqs

{'Chinstrap': array([0.31689883, 0.31237582, 0.20405537, 0.24846588]),
 'Gentoo': array([0.28810086, 0.25004371, 0.27702916, 0.40919054]),
 'Adelie': array([0.23452725, 0.4081271 , 0.20267081, 0.35176248])}

In [8]:
gnb.theta_

array([[-0.92625328,  0.52195557, -0.77406098, -0.59102787],
       [ 0.91764385,  0.70847529, -0.24527437, -0.53946387],
       [ 0.69991712, -1.13851399,  1.19942791,  1.12964089]])

In [9]:
mgnb.mus

{'Chinstrap': array([ 0.91764385,  0.70847529, -0.24527437, -0.53946387]),
 'Gentoo': array([ 0.69991712, -1.13851399,  1.19942791,  1.12964089]),
 'Adelie': array([-0.92625328,  0.52195557, -0.77406098, -0.59102787])}

Accuracy scores are the same:

In [10]:
gnb.score(X_test, y_test)

0.9640718562874252

In [11]:
mgnb.score(X_test, y_test)

0.9640718562874252

### Fidelity tests

Look at the lables of the predictions of the first 10 test samples:

In [12]:
mgnb.predict(X_test[:10])

array(['Chinstrap', 'Gentoo', 'Adelie', 'Gentoo', 'Adelie', 'Adelie',
       'Gentoo', 'Adelie', 'Adelie', 'Adelie'], dtype='<U9')

In [13]:
gnb.predict(X_test[:10])

array(['Chinstrap', 'Gentoo', 'Adelie', 'Gentoo', 'Adelie', 'Adelie',
       'Gentoo', 'Adelie', 'Adelie', 'Adelie'], dtype='<U9')

Run multiple tests

In [14]:
fidelity_tests(X_raw, y)

Run: 1 Score: 1.00 SK acc: 0.94 My acc: 0.94
Run: 2 Score: 1.00 SK acc: 0.96 My acc: 0.96
Run: 3 Score: 1.00 SK acc: 0.95 My acc: 0.95
Run: 4 Score: 1.00 SK acc: 0.95 My acc: 0.95
Run: 5 Score: 1.00 SK acc: 0.96 My acc: 0.96
Run: 6 Score: 1.00 SK acc: 0.96 My acc: 0.96
Run: 7 Score: 1.00 SK acc: 0.98 My acc: 0.98
Run: 8 Score: 1.00 SK acc: 0.97 My acc: 0.97
Run: 9 Score: 1.00 SK acc: 0.96 My acc: 0.96
Run: 10 Score: 1.00 SK acc: 0.99 My acc: 0.99


Finally, we use `accuracy_score` to compare all predictions on the test set.   
The score of 1.0 indicates perfect agreement.

### Penguins MV0.2 dataset

In [15]:
#Load the PenguinsMV0.2 dataset
penguinsMV2 = pd.read_csv('PenguinsMV0.2.csv', index_col = 0, na_values = '?')  #
print(penguinsMV2.shape)
penguinsMV2.head()

f_names = ['bill_length', 'bill_depth','flipper_length', 'body_mass']
penguinsMV2 = penguinsMV2[f_names + ['species']]

(333, 5)


In [16]:
y = penguinsMV2.pop('species').values
X = penguinsMV2.values
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.5,
                                                    random_state=42)
X_train.shape, X_test.shape

((166, 4), (167, 4))

### 1 Using MyGaussianNB

In [17]:
mGNBpipe  = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', MyGaussianNB())])

In [18]:
mgnb_param_grid = {
              'scaler':[StandardScaler(), MinMaxScaler(),'passthrough'],
             }
mgnb_pipe_gs = GridSearchCV(mGNBpipe,mgnb_param_grid,cv=10, 
                      verbose = 1, n_jobs = -1)
mgnb_pipe_gs = mgnb_pipe_gs.fit(X_train, y_train)


Fitting 10 folds for each of 3 candidates, totalling 30 fits


In [19]:
mgnb_pipe_gs.best_params_

{'scaler': StandardScaler()}

In [20]:
y_mgnb_pred_gs = mgnb_pipe_gs.predict(X_test)
print("MyGaussianNB, Accuracy: {0:4.2f}".format(accuracy_score(y_test,y_mgnb_pred_gs)))

MyGaussianNB, Accuracy: 0.94


### 2 Using GaussianNB

### 2-1 gnb  imputer: KNN

In [21]:
gNBpipe  = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=6, weights="uniform")),
    ('scaler', StandardScaler()),
    ('classifier', GaussianNB())])

#acc_arr = cross_val_score(gNBpipe, X, y, cv=9, n_jobs = -1)
#print("Accuracy: {0:4.2f}".format(sum(acc_arr)/len(acc_arr)))

In [22]:
gnb_param_grid = {
              'scaler':[StandardScaler(), MinMaxScaler(),'passthrough'],
             }
gnb_pipe_gs = GridSearchCV(gNBpipe,gnb_param_grid,cv=10, 
                      verbose = 1, n_jobs = -1)
gnb_pipe_gs = gnb_pipe_gs.fit(X_train, y_train)

Fitting 10 folds for each of 3 candidates, totalling 30 fits


In [23]:
gnb_pipe_gs.best_params_

{'scaler': StandardScaler()}

In [24]:
y_gnb_pred_gs = gnb_pipe_gs.predict(X_test)
print("Imputer: KNNImputer, Accuracy: {0:4.2f}".format(accuracy_score(y_test,y_gnb_pred_gs)))

Imputer: KNNImputer, Accuracy: 0.93


### 2-2 gnb SimpleImputer 

In [25]:
gNBpipe  = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values = np.nan)),
    ('scaler', StandardScaler()),
    ('classifier', GaussianNB())])

In [26]:
gnb_param_grid = {
              'scaler':[StandardScaler(), MinMaxScaler(),'passthrough'],
             }
gnb_pipe_gs = GridSearchCV(gNBpipe,gnb_param_grid,cv=10, 
                      verbose = 1, n_jobs = -1)
gnb_pipe_gs = gnb_pipe_gs.fit(X_train, y_train)

Fitting 10 folds for each of 3 candidates, totalling 30 fits


In [27]:
gnb_pipe_gs.best_params_

{'scaler': StandardScaler()}

In [28]:
y_gnb_pred_gs = gnb_pipe_gs.predict(X_test)
print("Imputer: SimpleImputer, Accuracy: {0:4.2f}".format(accuracy_score(y_test,y_gnb_pred_gs)))

Imputer: SimpleImputer, Accuracy: 0.93


### 2-3 gnb IterativeImputer

In [29]:
#Using multi-variate imputation algorithm
gNBpipe  = Pipeline(steps=[
    ('imputer', IterativeImputer(missing_values = np.nan)),
    ('scaler', StandardScaler()),
    ('classifier', GaussianNB())])

In [30]:
gnb_param_grid = {
              'scaler':[StandardScaler(), MinMaxScaler(),'passthrough'],
             }
gnb_pipe_gs = GridSearchCV(gNBpipe,gnb_param_grid,cv=10, 
                      verbose = 1, n_jobs = -1)
gnb_pipe_gs = gnb_pipe_gs.fit(X_train, y_train)

Fitting 10 folds for each of 3 candidates, totalling 30 fits


In [31]:
gnb_pipe_gs.best_params_

{'scaler': StandardScaler()}

In [32]:
y_gnb_pred_gs = gnb_pipe_gs.predict(X_test)
print("Imputer: IterativeImputer, Accuracy: {0:4.2f}".format(accuracy_score(y_test,y_gnb_pred_gs)))

Imputer: IterativeImputer, Accuracy: 0.95


Iterative Imputer performed best among the three imputers when GaussianNB classifier was used

### PenguinsMV0.4 dataset

In [33]:
#Load the PenguinsMV0.4 dataset
penguinsMV4 = pd.read_csv('PenguinsMV0.4.csv', index_col = 0, na_values = '?')  
print(penguinsMV4.shape)
penguinsMV4.head()

f_names = ['bill_length', 'bill_depth','flipper_length', 'body_mass']
penguinsMV4 = penguinsMV4[f_names + ['species']]

(333, 5)


In [34]:
y = penguinsMV4.pop('species').values
X = penguinsMV4.values
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.5,
                                                    random_state=42)
X_train.shape, X_test.shape

((166, 4), (167, 4))

### 1 Using MyGaussianNB

In [35]:
mGNBpipe  = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', MyGaussianNB())])

In [36]:
cross_validation_folds = 1
for num in range(1,10):
    cross_validation_folds=cross_validation_folds+1
    acc_arr = cross_val_score(mGNBpipe, X, y, cv=cross_validation_folds, n_jobs = -1)
    print('\n{0}-fold cross validation score ("Accuracy") : {1:.2f}'.format(cross_validation_folds,sum(acc_arr)/len(acc_arr)))


2-fold cross validation score ("Accuracy") : 0.86

3-fold cross validation score ("Accuracy") : 0.86

4-fold cross validation score ("Accuracy") : 0.86

5-fold cross validation score ("Accuracy") : 0.86

6-fold cross validation score ("Accuracy") : 0.87

7-fold cross validation score ("Accuracy") : 0.86

8-fold cross validation score ("Accuracy") : 0.87

9-fold cross validation score ("Accuracy") : 0.87

10-fold cross validation score ("Accuracy") : 0.87


### 2 Using GaussianNB

### 2-1 Using KNNImputer

In [37]:
gNBpipe  = Pipeline(steps=[
    ('imputer', KNNImputer(missing_values = np.nan)),
    ('scaler', StandardScaler()),
    ('classifier', GaussianNB())])

In [38]:
cross_validation_folds = 1
for num in range(1,10):
    cross_validation_folds=cross_validation_folds+1
    acc_arr = cross_val_score(gNBpipe, X, y, cv=cross_validation_folds, n_jobs = -1)
    print('\n{0}-fold cross validation score ("Accuracy") : {1:.2f}'.format(cross_validation_folds,sum(acc_arr)/len(acc_arr)))


2-fold cross validation score ("Accuracy") : 0.81

3-fold cross validation score ("Accuracy") : 0.82

4-fold cross validation score ("Accuracy") : 0.81

5-fold cross validation score ("Accuracy") : 0.80

6-fold cross validation score ("Accuracy") : 0.81

7-fold cross validation score ("Accuracy") : 0.82

8-fold cross validation score ("Accuracy") : 0.81

9-fold cross validation score ("Accuracy") : 0.81

10-fold cross validation score ("Accuracy") : 0.81


### 2-2 Using SimpleImputer

In [39]:
gNBpipe = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values = np.nan)),
    ('scaler', StandardScaler()),
    ('classifier', GaussianNB())])

In [40]:
cross_validation_folds = 1
for num in range(1,10):
    cross_validation_folds=cross_validation_folds+1
    acc_arr = cross_val_score(gNBpipe, X, y, cv=cross_validation_folds, n_jobs = -1)
    print('\n{0}-fold cross validation score ("Accuracy") : {1:.2f}'.format(cross_validation_folds,sum(acc_arr)/len(acc_arr)))


2-fold cross validation score ("Accuracy") : 0.83

3-fold cross validation score ("Accuracy") : 0.83

4-fold cross validation score ("Accuracy") : 0.84

5-fold cross validation score ("Accuracy") : 0.83

6-fold cross validation score ("Accuracy") : 0.84

7-fold cross validation score ("Accuracy") : 0.84

8-fold cross validation score ("Accuracy") : 0.85

9-fold cross validation score ("Accuracy") : 0.85

10-fold cross validation score ("Accuracy") : 0.85


### 2-3 Using IterativeImputer

In [41]:
gNBpipe  = Pipeline(steps=[
    ('imputer', IterativeImputer(missing_values = np.nan)),
    ('scaler', StandardScaler()),
    ('classifier', GaussianNB())])

In [42]:
cross_validation_folds = 1
for num in range(1,10):
    cross_validation_folds=cross_validation_folds+1
    acc_arr = cross_val_score(gNBpipe, X, y, cv=cross_validation_folds, n_jobs = -1)
    print('\n{0}-fold cross validation score ("Accuracy") : {1:.2f}'.format(cross_validation_folds,sum(acc_arr)/len(acc_arr)))


2-fold cross validation score ("Accuracy") : 0.84

3-fold cross validation score ("Accuracy") : 0.84

4-fold cross validation score ("Accuracy") : 0.84

5-fold cross validation score ("Accuracy") : 0.83

6-fold cross validation score ("Accuracy") : 0.84

7-fold cross validation score ("Accuracy") : 0.83

8-fold cross validation score ("Accuracy") : 0.84

9-fold cross validation score ("Accuracy") : 0.83

10-fold cross validation score ("Accuracy") : 0.84


## Comments on the performance

 1. Among the four imputers, IterativeImputer performed best when using Penguins MV0.2 dataset

2. Among the four imputers, MyGussianNB performed best of filling in missing values when using Penguins MV0.4 dataset. Otherwise, among the three imputers using in the GaussianNB classifier, SimpleImputer performed best.